# Anti-Hater Filter for Social Networks

Develop an automated moderation system based on Deep Learning techniques capable of identifying and filtering toxic, offensive, or inappropriate comments in real-time within an online platform.
The model should classify comments into multiple relevant categories, ensuring effective management of harmful content without compromising the user experience.

## Import libraries

In [ ]:
!pip uninstall -y tensorflow

In [ ]:
!pip install tensorflow==2.15

In [ ]:
import tensorflow as tf
print(tf.__version__)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import numpy as np

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import warnings
warnings.filterwarnings("ignore")
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import Input, Embedding, Conv1D, Dropout, MaxPooling1D, GRU, Bidirectional, concatenate, Dense, BatchNormalization
from keras.backend import clear_session
from keras.models import load_model
import tensorflow_addons as tfa
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split



from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import re
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt_tab')

## Import dataset and pre-processing

Textual comments must be converted into numerical sequences (tokenization).

The data must be normalized and balanced to ensure that all categories of toxicity are represented equally.

In [ ]:
BASE_URL = "https://s3.eu-west-3.amazonaws.com/profession.ai/datasets/"
df_bck = pd.read_csv(BASE_URL+"Filter_Toxic_Comments_dataset.csv")

In [ ]:
df_bck.head()
df_bck.shape

The dataset contains 159,571 comments, with each comment labeled in one or more categories. Comments may have zero or more active labels.

The six categories are:

1) Toxic
2) Severely Toxic
3) Obscene
4) Threat
5) Insult
6) Identity Hate

In [ ]:
df =df_bck.copy()

Preprocessing of the text  by removing URLs, HTML tags, non-alphabetic characters, tokenizing, removing stopwords, and lemmatizing the words. The second stage of the text preprocessing involves instead the removal of the stopwords and the lemmatization of each token in both train and test sets:
- Stop words are words that are commonly used but do not carry much meaning on their own, such as "the" and "and". Removing these words can help to simplify the text and improve model performance.
- lemmatization, insetad, is the process of reducing words to their base form (or lemma). For example, the word "running" could be reduced to its base form "run". Lemmatization can help to normalize the text and reduce the number of unique words that a model needs to learn.

To accomplish these two operations I exploited the `NLTK` library:

- The `stopwords` module from NLTK contains a list of common English stop words that can be removed from text during preprocessing.
- The `WordNetLemmatizer` class from NLTK is used to perform lemmatization

The preprocess_text function tokenizes the input text using word_tokenize from NLTK. It then removes stop words using the stopwords module and lemmatizes the remaining tokens using WordNetLemmatizer. Finally, it joins the lemmatized tokens back into a string and returns the preprocessed text.

Furthermore removing rows from the DataFrame where the column contains only spaces or is empty.

In [ ]:
def preprocess_text(text):
    """
    Preprocesses the given text by removing URLs, HTML tags, non-alphabetic characters,
    tokenizing, removing stopwords, and lemmatizing the words.

    :param text: A string containing the text to be preprocessed.
    :return: A string of the preprocessed text.
    """
    # Remove URLs
    text = re.sub(r'http\S+', '', text)
    # Remove HTML tags
    text = re.sub(r'<.*?>', '', text)
    # Remove non-alphabetic characters
    text = re.sub(r'[^a-zA-Z\s]', '', text, re.I|re.A)

    # Tokenize
    tokens = word_tokenize(text)

    # Remove stopwords and lemmatize
    stop_words = set(stopwords.words('english'))
    lemmatizer = WordNetLemmatizer()
    cleaned_tokens = [lemmatizer.lemmatize(word.lower()) for word in tokens if word.lower() not in stop_words]

    # Reconstruct the preprocessed text
    cleaned_text = ' '.join(cleaned_tokens)

    return cleaned_text

def remove_empty_or_space_only_rows(data):
    """
    Removes rows from the DataFrame where the 'cleaned_comment_text' column contains only spaces or is empty.

    :param data: A pandas DataFrame expected to contain a column named 'cleaned_comment_text'.
    :return: A tuple containing two DataFrames:
             1. The rows that were removed (having only spaces or being empty in 'cleaned_comment_text').
             2. The cleaned DataFrame with these rows removed.
    """
    # Identify rows with 'cleaned_comment_text' that are either empty or contain only spaces
    rows_with_only_spaces = data[data['cleaned_comment_text'].apply(lambda x: x.isspace() or not x)]

    # Remove these rows from the dataset
    data = data[~data['cleaned_comment_text'].apply(lambda x: x.isspace() or not x)]

    return data

In [ ]:
df['cleaned_comment_text'] = df['comment_text'].apply(preprocess_text)
df[['comment_text', 'cleaned_comment_text']].head()
remove_empty_or_space_only_rows(df)

In [ ]:
df = df.drop(columns=['sum_injurious'])

Verifing the categories contain only 0 and 1

In [ ]:
for col in df.columns[1:-1]:
    print(f"Column '{col}': {df[col].unique()}")

In [ ]:
categories = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
distribution = df[categories].sum().sort_values()

In [ ]:
# Initialize the plot
plt.figure(figsize=(14, 6))

# Plot the distribution of comments per category
plt.subplot(1, 2, 1)
sns.barplot(x=distribution.values, y=distribution.index)
plt.title('Distribution of Comments by Category')
plt.xlabel('Number of Comments')
plt.ylabel('Category')
# Adjust the layout and display the plot
plt.tight_layout()
plt.show()

The distribution of comments by category shows that most comments are classified as non-harmful in all categories. However, the toxic category has the highest number of occurrences, followed by obscene and insult. The severe_toxic, threat and identity_hate categories have significantly fewer comments.

In [ ]:
# Calculate the length of each comment
df['comment_length'] = df['cleaned_comment_text'].apply(len)
# Calculate the total number of damaging vs non-damaging comments
df['is_harmful'] = df[categories].sum(axis=1) > 0
harmful_distribution = df['is_harmful'].value_counts()

# Calculate the length of harmful vs non-harmful comments
harmful_comment_length = df[df['is_harmful']]['comment_length']
non_harmful_comment_length = df[~df['is_harmful']]['comment_length']

# Initialize the plot
plt.figure(figsize=(18, 5))

# Plot the distribution of harmful vs non-harmful comments
plt.subplot(1, 3, 1)
sns.barplot(x=harmful_distribution.index, y=harmful_distribution.values, palette='viridis')
plt.title('Distribution of Harmful vs Non-Harmful Comments')
plt.xlabel('harmful')
plt.ylabel('Number of Comments')
plt.xticks([0, 1], ['Non-harmful', 'harmful'])

# Plot the distribution of lengths for harmful comments
plt.subplot(1, 3, 2)
sns.histplot(harmful_comment_length, bins=50, color='red', label='Damaging')
plt.title('Length of harmful Comments')
plt.xlabel('Comment Length')
plt.ylabel('Frequency')
plt.xlim(0, 2000)

# Plot the distribution of lengths for non-harmful comments
plt.subplot(1, 3, 3)
sns.histplot(non_harmful_comment_length, bins=50, color='green', label='Non-harmful')
plt.title('Length of Non-harmful Comments')
plt.xlabel('Comment Length')
plt.ylabel('Frequency')
plt.xlim(0, 2000)

# Adjust the layout and display the plot
plt.tight_layout()
plt.show()

The distribution of comments in the dataset highlights a clear imbalance, with non-harmful comments significantly outnumbering harmful ones. Approximately 140,000 comments are classified as non-harmful, while only around 20,000 are labeled as harmful. This disparity suggests that the majority of the dataset consists of non-harmful content, with harmful comments representing a relatively small portion of the data.

Analyzing the lengths of harmful comments reveals that they are predominantly short, with most falling within the 0-250 character range. The frequency of harmful comments decreases sharply as their length increases, and very few exceed 750 characters. The distribution is notably right-skewed, emphasizing the brevity of harmful content.

Similarly, non-harmful comments also tend to be short, with the majority concentrated in the 0-250 character range. However, the decline in frequency as comment length increases is more gradual compared to harmful comments. Non-harmful comments have a longer tail in their distribution, indicating a greater presence of longer comments. While the distribution is also right-skewed, it shows more variation in length, with more substantial frequencies in the middle ranges.

In summary, the dataset predominantly consists of shorter comments for both harmful and non-harmful categories. However, non-harmful comments exhibit a greater range of lengths, suggesting more variability in their content. Despite the imbalance in the overall distribution, the data effectively captures the general characteristics of both types of comments.

## Text Data Augmentation

To address class imbalance in the toxic comment classification dataset, i implemented several text augmentation techniques. The augmentation process was carefully designed to maintain the semantic meaning of the original texts while creating diverse variations of the minority class samples.

1. Synonym Replacement

Randomly selects words from the text and replaces them with their synonyms
Uses WordNet from NLTK to find appropriate synonyms
Preserves the original meaning while introducing lexical variation

2. Random Insertion

Identifies synonyms of random words in the text
Inserts these synonyms at random positions
Increases text length while maintaining semantic coherence

3. Random Swap

Randomly selects pairs of words in the text
Swaps their positions
Creates syntactic variations while preserving all original words

4. Random Deletion

Randomly removes words with a specified probability
Helps create shorter variations of the text
Maintains core meaning while varying text length

In [ ]:
import nltk
from nltk.corpus import wordnet
import random
import numpy as np
from collections import defaultdict
import pandas as pd
from sklearn.model_selection import train_test_split

def get_synonyms(word):
    """Get synonyms of a word using WordNet."""
    synonyms = []
    for syn in wordnet.synsets(word):
        for lemma in syn.lemmas():
            if lemma.name() != word and '_' not in lemma.name():
                synonyms.append(lemma.name())
    return list(set(synonyms))

def synonym_replacement(text, n=1):
    """Replace n random words with their synonyms."""
    words = text.split()
    new_words = words.copy()
    random_word_list = list(set([word for word in words if len(get_synonyms(word)) > 0]))

    n = min(n, len(random_word_list))
    if n == 0:
        return text

    for _ in range(n):
        random_word = random.choice(random_word_list)
        random_synonym = random.choice(get_synonyms(random_word))
        random_idx = random.randrange(len(new_words))
        new_words[random_idx] = random_synonym

    return ' '.join(new_words)

def random_insertion(text, n=1):
    """Insert n random synonyms of random words in the text."""
    words = text.split()
    new_words = words.copy()
    for _ in range(n):
        add_word = random.choice(words)
        synonyms = get_synonyms(add_word)
        if synonyms:
            random_synonym = random.choice(synonyms)
            random_idx = random.randrange(len(new_words))
            new_words.insert(random_idx, random_synonym)
    return ' '.join(new_words)

def random_swap(text, n=1):
    """Randomly swap the positions of n pairs of words."""
    words = text.split()
    new_words = words.copy()
    for _ in range(n):
        if len(new_words) >= 2:
            idx1, idx2 = random.sample(range(len(new_words)), 2)
            new_words[idx1], new_words[idx2] = new_words[idx2], new_words[idx1]
    return ' '.join(new_words)

def random_deletion(text, p=0.1):
    """Randomly delete words from the text with probability p."""
    words = text.split()
    if len(words) == 1:
        return text
    new_words = []
    for word in words:
        if random.random() > p:
            new_words.append(word)
    if len(new_words) == 0:
        return random.choice(words)
    return ' '.join(new_words)

def augment_text(text, num_augmentations=4):
    """Apply multiple augmentation techniques to generate new samples."""
    augmentation_functions = [
        synonym_replacement,
        random_insertion,
        random_swap,
        lambda x: random_deletion(x, p=0.1)
    ]

    augmented_texts = []
    for _ in range(num_augmentations):
        # Randomly choose an augmentation function
        aug_func = random.choice(augmentation_functions)
        augmented_texts.append(aug_func(text))

    return augmented_texts

def balance_dataset(df, categories, num_augmentations=2):
    """Balance the dataset by augmenting minority classes."""
    # Initialize dictionary to store balanced data
    balanced_data = {
        'text': [],
        **{cat: [] for cat in categories}
    }

    # Find the maximum number of samples in any category
    max_samples = max(df[categories].sum())

    # Process each category
    for category in categories:
        # Get positive samples for current category
        positive_mask = df[category] == 1
        positive_samples = df[positive_mask]['cleaned_comment_text'].values
        current_samples = len(positive_samples)

        # If we need more samples for this category
        if current_samples < max_samples:
            samples_needed = max_samples - current_samples
            augmented_samples = []

            # Generate augmented samples
            while len(augmented_samples) < samples_needed:
                text = random.choice(positive_samples)
                new_texts = augment_text(text, num_augmentations=1)  # Generate one augmentation at a time
                augmented_samples.extend(new_texts[:min(1, samples_needed - len(augmented_samples))])

            # Add original samples
            balanced_data['text'].extend(positive_samples)
            for cat in categories:
                balanced_data[cat].extend([1 if cat == category else 0] * len(positive_samples))

            # Add augmented samples
            balanced_data['text'].extend(augmented_samples)
            for cat in categories:
                balanced_data[cat].extend([1 if cat == category else 0] * len(augmented_samples))

    # Convert to DataFrame
    balanced_df = pd.DataFrame(balanced_data)
    return balanced_df

def prepare_augmented_data(df, categories):
    """Prepare the data with augmentation only on training set."""
    # First split for test set
    X = df['cleaned_comment_text']
    y = df[categories]

    X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.11, random_state=42)

    # Second split for validation set
    X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.10, random_state=42)

    # Create temporary DataFrame for training set
    train_df = pd.DataFrame({
        'cleaned_comment_text': X_train.reset_index(drop=True),
        **{cat: y_train[cat].reset_index(drop=True) for cat in categories}
    })

    # Apply data augmentation only to training set
    balanced_train_df = balance_dataset(train_df, categories)

    # Merge original training data with augmented data
    augmented_train_df = pd.concat([train_df, balanced_train_df], ignore_index=True)

    # Prepare final data
    X_train_aug = augmented_train_df['text'] if 'text' in augmented_train_df.columns else augmented_train_df['cleaned_comment_text']
    y_train_aug = augmented_train_df[categories]

    return (X_train_aug, X_val, X_test), (y_train_aug, y_val, y_test)

In [ ]:
# Definire le categorie
categories = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

# Ottenere i set di dati con augmentation solo sul training set
(X_train, X_val, X_test), (Y_train, Y_val, Y_test) = prepare_augmented_data(df, categories)

In [ ]:
print(f"Train set size: {len(X_train)}")
print(f"Validation set size: {len(X_val)}")
print(f"Test set size: {len(X_test)}")

Inverse class weights are calculated based on the class frequencies in the training set. This approach addresses class imbalance by assigning higher weights to underrepresented classes, ensuring they have a greater influence during model training.

In [ ]:
def calculate_class_weights(y_train):
    """
    Calculates inverse class weights based on the frequencies of classes in the training set. This is often used to
    handle imbalanced datasets by giving more weight to underrepresented classes during model training.

    :param y_train: A pandas DataFrame or numpy array containing the training labels.
                    Each column represents a class, and each row represents a sample with binary indicators for class membership.
    :return: A dictionary with class indices as keys and calculated inverse weights as values.
    """
    # Calculate the frequency of each class
    class_frequencies = y_train.sum(axis=0)

    # Calculate inverse weights for each class
    weights = [len(y_train) / (len(class_frequencies) * frequency) if frequency > 0 else 0 for frequency in class_frequencies]

    # Create a dictionary mapping class index to its weight
    class_weights = {i: weight for i, weight in enumerate(weights)}

    # Optionally, you can print the calculated weights
    #print("Calculated weights for classes:", class_weights)

    return class_weights

In [ ]:
class_weights = calculate_class_weights(Y_train)

In [ ]:
print(class_weights)

### Text Vecorization

***Important preprocessing step when working with text data in machine learning because it allows the text data to be transformed into numerical data that can be used as input to machine learning models.***

- One common technique for vectorizing text is through tokenization, which involves breaking the text into smaller units called tokens, such as words or phrases. Once tokenized, these units can then be mapped to numerical values to create vector representations of the text.

- In this context, the Keras `Tokenizer` class provides a useful tool for tokenizing and vectorizing text data. By using the `fit_on_texts()` method of the Tokenizer object, the internal vocabulary of the tokenizer can be updated based on the frequency of each word in the text data.

- It is important to **fit the tokenizer only on the training set** because we want to learn the vocabulary and the word index based on the training data. If we fit the tokenizer also on the test set, we would be introducing bias into our model. The model would have knowledge of the test set and could potentially overfit to it, resulting in poorer performance on unseen data.

- After fitting the tokenizer on the training data, we can use `texts_to_sequences()` method to convert the texts, in both the training, validation and test data, to sequences of integers representing the token indices of the words in the original text. This ensures that the same vocabulary is used for both the training and test data, which is important for the model to generalize well to new data.

In [ ]:
# Convert all values to strings and handle NaN values
def clean_text_data(text_series):
    # Convert to string and replace NaN with empty string
    return text_series.fillna('').astype(str)

# Clean all text datasets
X_train = clean_text_data(X_train)
X_val = clean_text_data(X_val)
X_test = clean_text_data(X_test)

In [ ]:
X_tokenizer = Tokenizer()
X_tokenizer.fit_on_texts(X_train)

#sequences
X_sequences_train = X_tokenizer.texts_to_sequences(X_train)
X_sequences_val = X_tokenizer.texts_to_sequences(X_val)
X_sequences_test = X_tokenizer.texts_to_sequences(X_test)

### Padding

**Padding involves adding zeros or other placeholder values to the end of shorter sequences to make all sequences in a dataset of equal length, either by padding them to the length of the longest sequence in the dataset or to a fixed length specified by hand. This is important because deep learning model such as Recurrent Neural Networks (RNNs) or Transformers (e.g. BERT) require inputs of a fixed size. Padding ensures that all sequences have the same length, allowing these models to process them in a consistent manner. ***

- typically need to use the `pad_sequences()` function from Keras to pad the sequences to a fixed length.

- However, the fixed length can be considerd as a hyperparamter and we need to choose carefully this value.

In [ ]:
maxlen = len(max(X_sequences_train,key=len))

#train padding
padded_X_sequences_train = pad_sequences(X_sequences_train, padding = 'pre', maxlen = maxlen)

#val padding
padded_X_sequences_val = pad_sequences(X_sequences_val, padding = 'pre', maxlen = maxlen)

# test padding
padded_X_sequences_test = pad_sequences(X_sequences_test, padding='pre', maxlen=maxlen)

In [ ]:
# Dimesnion Check
print(f"Train sequences shape: {padded_X_sequences_train.shape}")
print(f"Validation sequences shape: {padded_X_sequences_val.shape}")
print(f"Test sequences shape: {padded_X_sequences_test.shape}")

In [ ]:
vocab_size = len(X_tokenizer.word_index)+1
print("Max sentence length: {}".format(padded_X_sequences_train.shape[1]))
print("vocabulary size: {}".format(vocab_size))

# Model

I use as model the multichannel convolutional bidirectional gated recurrent unit for multilabel toxic category detection proposed by Kumar et al. (2021) [1].

This model extends the standard CNN by using multiple channels with different kernel sizes, enabling the simultaneous processing of various n-grams (e.g., 1-gram, 2-gram, 3-grams). The architecture includes a word embedding layer, a 1D-convolutional layer, dropout, max-pooling, a bidirectional GRU layer, and another dropout layer. It is designed to capture diverse linguistic patterns effectively across different channels.

1. Multichannel Word Embedding
Words are preprocessed by removing punctuation and special symbols, then converted into numeric vectors using pre-trained GloVe embeddings (100 dimensions). In the multichannel setup, word embeddings are generated with varying contexts or window sizes for each channel. This enables parallel extraction of features from the same training data. Notably, the embeddings are not updated during training, ensuring consistency in semantic representation.

2. Convolutional Neural Network (CNN)
The CNN layer applies multiple filters over the input sequence to generate feature maps. Each channel uses a 1D-CNN with a set of filters and varying kernel sizes, producing new feature maps by scanning the input sequence. These maps capture local patterns, forming the basis for downstream processing.

3. Pooling Layers
Pooling layers reduce the dimensions of feature maps to focus on the most important features. Maximum pooling is applied across channels, selecting the maximum value from local neighborhoods of each feature map. This step ensures dimensionality reduction while retaining key information.

4. Bidirectional Gated Recurrent Unit (GRU)
The bidirectional GRU layer processes input sequences in both forward and backward directions, capturing semantic information from both past and future contexts. GRUs are computationally more efficient than LSTMs and rely on update and reset gates to manage memory. When combined with CNN and pooling layers, this architecture effectively models sequential dependencies while reducing computational complexity.

5. Output Layer
The outputs from all channels are concatenated and passed through dense and normalization layers. Dense layers adjust vector dimensions to optimize trainable parameters, while normalization layers ensure efficient learning at each network layer. Finally, a sigmoid activation function in the output layer predicts multilabel categories using binary cross-entropy loss to determine whether a comment belongs to one or more categories.

![FlowChart.jpg](attachment:FlowChart.jpg)

![Hyperparameters.jpg](attachment:Hyperparameters.jpg)

### Multichannel Word Embedding Words

In [ ]:
embeddings_index = {}
with open('glove.6B.100d.txt', encoding='utf8') as f:
    for line in tqdm(f, desc="Loading GloVe embeddings"):
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
print('GloVe data loaded')

In [ ]:
embedding_dim = 100

embedding_matrix = np.zeros((vocab_size, embedding_dim))

for word, index in X_tokenizer.word_index.items():
    if index < vocab_size:
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:

            embedding_matrix[index] = embedding_vector
        else:

            embedding_matrix[index] = np.random.normal(scale=0.6, size=(embedding_dim,))


### Creation of the model

In [ ]:
clear_session()
embedding_dim = 100
filters = 128
kernel_sizes = [1, 2, 3, 5, 6]
gru_units = 200
dropout_rate = 0.6
pool_size = 4
output_units = 6

# Input Layer
input_layer = Input(shape=(maxlen,))

branches = []
for kernel_size in kernel_sizes:
    # Embedding layer
    embedding = Embedding(input_dim=vocab_size,
                          output_dim=embedding_dim,
                          weights=[embedding_matrix],
                          input_length=maxlen,
                          trainable=False)(input_layer)

    # 1D CNN
    cnn = Conv1D(filters=filters,
                 kernel_size=kernel_size,
                 activation='relu')(embedding)

    # Dropout
    cnn = Dropout(dropout_rate)(cnn)

    # Max Pooling
    pooled = MaxPooling1D(pool_size=pool_size)(cnn)

    # BiGRU
    bigru = Bidirectional(GRU(gru_units, return_sequences=False))(pooled)

    # Dropout
    bigru = Dropout(dropout_rate)(bigru)

    branches.append(bigru)

# Concatentation
concatenated = concatenate(branches)

# Fully Connected Layer
dense = Dense(128, activation='relu')(concatenated)

# Batch Normalization
normalized = BatchNormalization()(dense)

# Output Layer
output_layer = Dense(output_units, activation='sigmoid')(normalized)

model = Model(inputs=input_layer, outputs=output_layer)

# Compilation
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics = [tfa.metrics.F1Score(num_classes=6, average='macro', threshold=0.5)])
              #metrics=['accuracy'])

# Summary model
model.summary()

For the hyperparameters, reference was made to the table extracted from the article. Only two modifications were made: the number of epochs was set to 2 (unlike the article, which specifies 100), and the batch size was set to 216. This choice was driven by the computational resources available.

In [ ]:
history = model.fit(padded_X_sequences_train, Y_train,
                    validation_data=(padded_X_sequences_val, Y_val),
                    epochs=2,
                    batch_size=216,
                    class_weight= class_weights)

In [ ]:
# To save the model
model.save('model.h5')

In [ ]:
#To load the model
model = tf.keras.models.load_model('model.h5')

## Evaluating the model

In [ ]:
# Plot the loss and validation loss
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
# Plot the F1 macro score on the training and validation sets
plt.plot(history.history['f1_score'], label='Training F1')
plt.plot(history.history['val_f1_score'], label='Validation F1')
plt.xlabel('Epoch')
plt.ylabel('F1 score Score')
plt.legend()
plt.show()

In [ ]:
# Prediction
y_pred = model.predict(padded_X_sequences_test)

test_loss, test_f1_score = model.evaluate(padded_X_sequences_test, Y_test)
print(f"Test Loss: {test_loss}")
print(f"Test f1 score: {test_f1_score}")

In [ ]:
# Binarize the predictions using a threshold (default is 0.5)
y_pred_binary = (y_pred > 0.5).astype(int)

# Ensure the data is in NumPy array format
Y_test = np.array(Y_test)
y_pred_binary = np.array(y_pred_binary)

# Set up the layout for the figure (3 columns x 2 rows)
fig, axes = plt.subplots(2, 3, figsize=(15, 10))  # 2 rows, 3 columns
axes = axes.flatten()  # Flatten the array for easier indexing

# Compute and visualize the confusion matrix for each label
for i in range(min(6, Y_test.shape[1])):  # Assume a maximum of 6 labels
    print(f"Confusion matrix for label {i+1}:")

    # Compute the confusion matrix
    cm = confusion_matrix(Y_test[:, i], y_pred_binary[:, i])

    # Display the confusion matrix in the corresponding subplot
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["0", "1"])
    disp.plot(ax=axes[i], cmap=plt.cm.Blues)
    axes[i].set_title(f"Label {i+1}")

# Add a main title to the figure
plt.suptitle("Confusion Matrices for Each Label", fontsize=16)

# Optimize the layout
plt.tight_layout(rect=[0, 0, 1, 0.96])  # Create space for the main title

# Save the image
plt.savefig("confusion_matrices_all_labels.png")
plt.show()




# References

1) Kumar, A., Abirami, S., Trueman, T. E., & Cambria, E. (2021). Comment toxicity detection via a multichannel convolutional bidirectional gated recurrent unit. Neurocomputing, 441, 272-278.